In [ ]:
# =======================================
# CÉLULA 1 - Benchmark no PC
# =======================================
import platform
import time
import uuid
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from bot import Bot
from core.think import DodgeStrategy
from ultralytics import YOLO

MODEL_PATH = "../runs/detect/train/weights/best.pt"
RESULTS_DIR = Path("../resultados")
RESULTS_DIR.mkdir(exist_ok=True)
OUTPUT_FILE = RESULTS_DIR / "benchmark_hw.parquet"

# Identificação da máquina
machine_id = f"PC_{platform.node()}"

# -------------------------------
# Métrica 1: YOLOv5 - tempo médio de inferência
# -------------------------------
model = YOLO(MODEL_PATH)
img = None
if Path("test.png").is_file():
    img = cv2.imread("test.png")
if img is None:
    img = 255 * np.ones((640, 640, 3), dtype="uint8")

n_runs = 200
times = []
for _ in range(n_runs):
    t0 = time.perf_counter()
    _ = model(img)
    times.append(time.perf_counter() - t0)
avg_inference_s = sum(times) / len(times)

# -------------------------------
# Métrica 2: FPS simulado (dummy loop só do modelo)
# -------------------------------
n_frames = 200
t0 = time.perf_counter()
for _ in range(n_frames):
    _ = model(img)
fps_simulation = n_frames / (time.perf_counter() - t0)

# -------------------------------
# Métrica 3: Loop completo do Bot
# -------------------------------
bot = Bot(DodgeStrategy.MENOR_DISTANCIA)
metrics_bot = bot.benchmark(n_iters=200)

# -------------------------------
# Consolidação dos resultados
# -------------------------------
df_pc = pd.DataFrame([{
    "machine": machine_id,
    "avg_inference_s": avg_inference_s,       # YOLO puro
    "fps_simulation": fps_simulation,         # FPS YOLO dummy
    "avg_loop_s": metrics_bot["avg_loop_s"],  # tempo loop completo
    "avg_loop_ns": metrics_bot["avg_loop_ns"],
    "fps_loop": metrics_bot["fps_loop"],      # FPS loop completo
    "uuid": uuid.uuid4().hex,
}])

if OUTPUT_FILE.exists():
    df_old = pd.read_parquet(OUTPUT_FILE)
    df_all = pd.concat([df_old, df_pc], ignore_index=True)
else:
    df_all = df_pc

df_all.to_parquet(OUTPUT_FILE, index=False)
print("Resultados salvos com sucesso no parquet.")

In [ ]:
# =======================================
# CÉLULA 2 - Benchmark no Notebook
# =======================================
import platform
import time
import uuid
import pandas as pd
from pathlib import Path
import cv2
import numpy as np

from bot import Bot
from core.think import DodgeStrategy

MODEL_PATH = "../runs/detect/train/weights/best.pt"
RESULTS_DIR = Path("../resultados")
RESULTS_DIR.mkdir(exist_ok=True)
OUTPUT_FILE = RESULTS_DIR / "benchmark_hw.parquet"

machine_id = f"NOTEBOOK_{platform.node()}"

# YOLO inferência
model = YOLO(MODEL_PATH)
img = None
if Path("test.png").is_file():
    img = cv2.imread("test.png")
if img is None:
    img = 255 * np.ones((640, 640, 3), dtype="uint8")

n_runs = 200
times = []
for _ in range(n_runs):
    t0 = time.perf_counter()
    _ = model(img)
    times.append(time.perf_counter() - t0)
avg_inference_s = sum(times) / len(times)

# FPS simulado
n_frames = 200
t0 = time.perf_counter()
for _ in range(n_frames):
    _ = model(img)
fps_simulation = n_frames / (time.perf_counter() - t0)

# Loop completo
bot = Bot(DodgeStrategy.MENOR_DISTANCIA)
metrics_bot = bot.benchmark(n_iters=200)

# Consolidado
df_nb = pd.DataFrame([{
    "machine": machine_id,
    "avg_inference_s": avg_inference_s,
    "fps_simulation": fps_simulation,
    "avg_loop_s": metrics_bot["avg_loop_s"],
    "avg_loop_ns": metrics_bot["avg_loop_ns"],
    "fps_loop": metrics_bot["fps_loop"],
    "uuid": uuid.uuid4().hex,
}])

if OUTPUT_FILE.exists():
    df_old = pd.read_parquet(OUTPUT_FILE)
    df_all = pd.concat([df_old, df_nb], ignore_index=True)
else:
    df_all = df_nb

df_all.to_parquet(OUTPUT_FILE, index=False)
print("Resultados do notebook salvos no parquet.")

In [ ]:
# =======================================
# CÉLULA 3 - Comparativo formatado p/ LaTeX
# =======================================
import pandas as pd
from pathlib import Path

OUTPUT_FILE = Path("../resultados/benchmark_hw.parquet")
df = pd.read_parquet(OUTPUT_FILE)

# Agrega os valores médios por máquina
df_summary = df.groupby("machine").agg({
    "avg_inference_s": "mean",
    "fps_simulation": "mean",
    "avg_loop_s": "mean",
    "fps_loop": "mean"
}).reset_index()

print("Resumo comparativo:")
display(df_summary)

# Exporta em formato LaTeX
latex_table = df_summary.to_latex(
    index=False,
    float_format="%.3f",
    caption="Benchmark comparativo entre PC e Notebook",
    label="tab:benchmark_hw"
)

print("\nTabela pronta para LaTeX:\n")
print(latex_table)